In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow

In [ ]:
!pip install torch==1.5.1+cpu torchvision==0.6.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers

In [ ]:
nltk.download('punkt')

In [1]:
import nltk
import numpy as np
from keras.models import model_from_json
import pandas as pd
import glob
import os
import json

In [ ]:
nltk.download('punkt')

In [2]:
from ClinicalBERTEmbeddings import ClinicalBERTEmbeddings

In [3]:
mytext = "random text file for testing creation of embeddings:"

In [4]:
data = ClinicalBERTEmbeddings(mytext)

In [5]:
embeddings_padded = []

for j in data:
    pad_len = 3840 - len(j)
    if pad_len:
        embeddings_padded.append(np.append(j, np.zeros(pad_len*768, dtype=np.float16).reshape(pad_len, 768), axis=0))
    else:
        embeddings_padded.append(j)

In [6]:
embeddings_padded = np.float16(embeddings_padded)
embeddings_padded = np.array(embeddings_padded, dtype=np.float16)

# Load & Initialize the Models

# Adverse Model

In [7]:
PATH = 'C:/Users/brent/Downloads/'

In [8]:
json_file = open(PATH + 'models/models/adverse/adverse_num.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
adverse_model = model_from_json(loaded_model_json)
adverse_model.load_weights(PATH + "models/models/adverse/adverse_model.hdr")

In [9]:
adverse_labels = pd.read_csv(PATH + 'models/models/adverse/adverse_effect_labels.csv').columns[1:]

# Chapter Model

In [10]:
json_file = open(PATH + 'models/models/chapter/chapter_num.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
chapter_model = model_from_json(loaded_model_json)
chapter_model.load_weights(PATH + "models/models/chapter/chapter_model.hdr")

In [11]:
chapter_labels = pd.read_csv(PATH + 'models/models/chapter/chapter_label.csv').columns[1:]

# Sub Chapter Models

In [12]:
model_dict = {}

In [13]:
label_dict = {}

In [14]:
for file in glob.glob(PATH + "models/models/disease/*.json"):  
    baseName = os.path.basename(file)[6:-5]   
    json_file = open(file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_dict[baseName] = model_from_json(loaded_model_json)
    model_dict[baseName].load_weights(file[:-5]+"_4_embeddings1.hdr")
    labels = pd.read_csv(PATH + 'models/models/disease/chapter_labels/'+baseName).columns[1:]
    label_dict[baseName] = labels

# Predictions

In [15]:
chapter_pred = chapter_model.predict(embeddings_padded)[0]
chapter_dict = {key: value for (key, value) in zip(chapter_labels, chapter_pred)}

In [16]:
adverse_pred = adverse_model.predict(embeddings_padded)[0]
adverse_dict = {key: value for (key, value) in zip(adverse_labels, adverse_pred)}

In [17]:
disease_dict = {}
disease_dict['680_709'] = {}
for model in model_dict:
    model_pred = model_dict[model].predict(embeddings_padded)[0]
    disease_dict[model[:-4]] = {key: value for (key,value) in zip(label_dict[model], model_pred)}

In [21]:
output_dict = {"name": "Root", "value": 1, "children": 
 [{"name": "Adverse", "value": 1, "children": 
      [{"name": adv_key, "value": adv_value} for (adv_key, adv_value) in adverse_dict.items()]},
 {"name": "Chapter", "value": 1, "children": 
      [{"name": ch_key, "value": ch_value, "children": 
          [{"name": dis_key, "value": dis_value} for (dis_key, dis_value) in disease_dict[ch_key].items()]} for (ch_key, ch_value) in chapter_dict.items()]}]
}